In [51]:
import pprint as pp
import redis
import json
import copy
import datetime
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017')
db = client.abaco

## Conversion Stuff

In [52]:
def unix_to_dt(unix):
    return datetime.datetime.utcfromtimestamp(float(unix))

def docker_time_to_dt(docker_time):
    return datetime.datetime.strptime(docker_time.replace('Z', '')[:-1], "%Y-%m-%dT%H:%M:%S.%f")

In [53]:
# Mongo Conversion
# Helper functions that converts Mongo to new formatting
def convertLogs(base_list, executions):
    copied_list = copy.deepcopy(base_list)
    newList = []
    for actor in copied_list:
        aid = actor['_id']
        newDict = {'exp': actor['exp'], '_id': aid, 'logs': actor[aid]}
        newList.append(newDict)
        
    for log in newList:
        wanted_exec_id = log['_id']
        for execution in executions:
            if execution['id'] == wanted_exec_id:
                log['actor_id'] = execution['actor_id']
                log['tenant'] = execution['tenant']
                break
    return newList

def convertPermissions(base_list):
    copied_list = copy.deepcopy(base_list)
    newList = []
    for actor in copied_list:
        aid = actor['_id']
        newDict = {'_id': aid}
        newDict.update(actor[aid])
        newList.append(newDict)
    return newList

def convertExecutions(base_list):
    copied_list = copy.deepcopy(base_list)
    newList = []
    for actor_outer in copied_list:
        del actor_outer['_id']
        for aid, execution in actor_outer.items():
            for exec_id, exec_dict in execution.items():
                exec_dict['_id'] = f"{aid}_{exec_id}"
                if exec_dict.get('start_time'):
                    exec_dict['start_time'] = unix_to_dt(exec_dict['start_time'])
                if exec_dict.get('message_received_time'):
                    exec_dict['message_received_time'] = unix_to_dt(exec_dict['message_received_time'])
                if exec_dict.get('final_state'):
                    if exec_dict.get('final_state').get('StartedAt'):
                        exec_dict['final_state']['StartedAt'] = docker_time_to_dt(exec_dict['final_state']['StartedAt'])
                    if exec_dict.get('final_state').get('FinishedAt'):
                        exec_dict['final_state']['FinishedAt'] = docker_time_to_dt(exec_dict['final_state']['FinishedAt'])
                newList.append(exec_dict)
    return newList

def convertClients(base_list):
    copied_list = copy.deepcopy(base_list)
    newList = []
    for actor in copied_list:
        aid = actor['_id']
        newDict = {'_id': aid}
        newDict.update(actor[aid])
        newList.append(newDict)
    return newList


# Redis Conversion
# Helper functions that converts Redis to new formatting
def convertActors(base_list):
    copied_list = copy.deepcopy(base_list)
    for actor_dict in copied_list:
        if actor_dict.get('last_update_time'):
            actor_dict['last_update_time'] = unix_to_dt(actor_dict['last_update_time'])
        if actor_dict.get('create_time'):
            actor_dict['create_time'] = unix_to_dt(actor_dict['create_time'])
    return copied_list

def convertWorkers(base_list):
    copied_list = copy.deepcopy(base_list)
    newList = []
    for actor in copied_list:
        aid = actor.pop('_id')
        for worker_id, worker_dict in actor.items():
            worker_dict['_id'] = f"{aid}_{worker_id}"
            worker_dict['actor_id'] = aid
            if worker_dict.get('create_time'):
                worker_dict['create_time'] = unix_to_dt(worker_dict['create_time'])
            if worker_dict.get('last_health_check_time'):
                worker_dict['last_health_check_time'] = unix_to_dt(worker_dict['last_health_check_time'])
            newList.append(worker_dict)
    return newList

def convertNonces(base_list):
    copied_list = copy.deepcopy(base_list)
    for nonce in copied_list:
        for nonce_id, nonce_dict in nonce.items():
            if not nonce_id == "_id":
                if nonce_dict.get('last_use_time'):
                    nonce_dict['last_use_time'] = unix_to_dt(nonce_dict['last_use_time'])
                if nonce_dict.get('create_time'):
                    nonce_dict['create_time'] = unix_to_dt(nonce_dict['create_time'])
    return copied_list

# Redis Reader
# Helper function that reads a Redis DB into a JSON dict
def redis2dict(db):
    allDocs = []
    redisDB = redis.Redis(db=db, port=6379)
    for key in redisDB.scan_iter():
        key = key.decode('utf-8')
        jsonDict = json.loads(redisDB.get(key))
        jsonDict['_id'] = key
        allDocs.append(jsonDict)
    return allDocs

# Metrics database creation
def createMetrics(base_list):
    actor_dbids = []
    actor_total = 0
    execution_dbids = []
    execution_total = 0

    copied_list = copy.deepcopy(base_list)

    for actor in copied_list:
        actor_dbids.append(actor.pop('_id'))
        actor_total += 1
        for _, actor_inner in actor.items():
            for _, execution in actor_inner.items():
                execution_dbids.append(f'{execution["actor_id"]}_{execution["id"]}')
                execution_total += 1
                
    metrics = [{'_id': 'stats',
                'actor_total': actor_total,
                'actor_dbids': actor_dbids,
                'execution_total': execution_total,
                'execution_dbids': execution_dbids}]
    return metrics

### Grabbing base data

In [54]:
base_logs_store_json = list(db['1'].find({}))
base_permissions_store_json = list(db['2'].find({}))
base_executions_store_json = list(db['3'].find({}))
base_clients_store_json = list(db['4'].find({}))
base_actors_store_json = redis2dict('1')
base_workers_store_json = redis2dict('2')
base_nonce_store_json = redis2dict('3')
base_alias_store_json = redis2dict('4')
base_pregen_clients_json = redis2dict('5')

### The converting

In [55]:
converted_permissions = convertPermissions(base_permissions_store_json)
converted_executions = convertExecutions(base_executions_store_json)
converted_clients = convertClients(base_clients_store_json)
converted_actors = convertActors(base_actors_store_json)
converted_workers = convertWorkers(base_workers_store_json)
converted_nonces = convertNonces(base_nonce_store_json)
converted_aliases = base_alias_store_json
converted_pregen = base_pregen_clients_json
converted_logs = convertLogs(base_logs_store_json, converted_executions)
new_metrics = createMetrics(base_executions_store_json)

### The uploading

In [57]:
db_naming = {'n1': converted_logs,
             'n2': converted_permissions,
             'n3': converted_executions,
             'n4': converted_clients,
             'n5': converted_actors,
             'n6': converted_workers,
             'n7': converted_nonces,
             'n8': converted_aliases,
             'n9': converted_pregen,
             'n10': new_metrics}

In [425]:
# Convert and upload modified data
for db_name, db_data in db_naming.items(): 
    try:
        print(f"Currently processing db: {db_name}")
        db[db_name].drop()
        db[db_name].insert_many(db_data)
    except TypeError:
        pass

Currently processing db: n1
Currently processing db: n2
Currently processing db: n3
Currently processing db: n4
Currently processing db: n5
Currently processing db: n6
Currently processing db: n7
Currently processing db: n8
Currently processing db: n9


# Comparison Stuff Functions

In [301]:
def issueRaise(new, old, words):
    print(f"Issue - {words}")
    pprint(new)
    print("=======================================")
    pprint(old)
    raise ValueError(words)

def increment(store, key):
    if key in store:
        store[key] += 1
    else:
        store[key] = 1

def incrementVals(store, key, val):
    if key in store:
        store[key].append(val)
    else:
        store[key] = [val]
        
def lenAndValCheck(inpDict):
    try:  
        keyStore = {}
        valStore = {}
        for i, actor in enumerate(inpDict):
                for worker in actor:
                    if worker == '_id':
                        increment(keyStore, '#actor_ids')
                        incrementVals(valStore, '#actor_ids', actor['_id'])
                    else:
                        for key, val in actor[worker].items():
                            increment(keyStore, key)
                            incrementVals(valStore, key, val)
    except AttributeError:
        keyStore = {}
        valStore = {}
        for i, actor in enumerate(inpDict):
                for key, val in actor.items():
                    increment(keyStore, key)
                    incrementVals(valStore, key, val)
    return keyStore, valStore

In [28]:
# Note: Convert old_coll to new stylings and use that as input.
def conversionChecker(new, old, debug=False):
    newKeys, newVals = lenAndValCheck(new)
    oldKeys, oldVals = lenAndValCheck(old)
    if debug:
        print(f'New Keys:\n{newKeys}\n Old Keys:\n{oldKeys}')
    for key in newKeys:
        try:
            if not newKeys[key] == oldKeys[key]:
                issueRaise(newVals[key], oldVals[key], f"Value lengths are not the same. {newKeys[key]} and {oldKeys[key]}.")
        except KeyError:
            raise KeyError(f"Key: {key} not found in old JSON.")
    for key in oldKeys:
        try:
            if not newKeys[key] == oldKeys[key]:
                issueRaise(newVals[key], oldVals[key], f"Value lengths are not the same. {newKeys[key]} and {oldKeys[key]}.")
        except KeyError:
            raise KeyError(f"Key: '{key}' not found in new JSON.")

## New

In [206]:
new_logs_store_json = list(db['1'].find({}))
new_permissions_store_json = list(db['2'].find({}))
new_executions_store_json = list(db['3'].find({}))
new_clients_store_json = list(db['4'].find({}))
new_actors_store_json = list(db['5'].find({}))
new_workers_store_json = list(db['6'].find({}))
new_nonce_store_json = list(db['7'].find({}))
new_alias_store_json = list(db['8'].find({}))
new_pregen_clients_json = list(db['9'].find({}))

## Old

In [ ]:
# Get JSON for each Redis store
old_logs_store_json = list(db['1'].find({}))
old_permissions_store_json = list(db['2'].find({}))
old_executions_store_json = list(db['3'].find({}))
old_clients_store_json = list(db['4'].find({}))
old_actors_store_json = redis2dict('1')
old_workers_store_json = redis2dict('2')
old_nonce_store_json = redis2dict('3')
old_alias_store_json = redis2dict('4')
old_pregen_clients_json = redis2dict('5')

## Comparisons

In [290]:
conversionChecker(new_logs_store_json, updateMongoLogs(old_logs_store_json))

KeyError: "Key: 'exp' not found in new JSON."

In [291]:
conversionChecker(new_permissions_store_json, updateMongoPermissions(old_permissions_store_json))

In [292]:
conversionChecker(new_executions_store_json, updateMongoExecutions(old_executions_store_json))

In [293]:
conversionChecker(new_actors_store_json, old_actors_store_json)

In [294]:
conversionChecker(new_workers_store_json, old_workers_store_json)

In [295]:
conversionChecker(new_nonce_store_json, old_nonce_store_json)

In [296]:
conversionChecker(new_alias_store_json, old_alias_store_json)

In [297]:
conversionChecker(new_pregen_clients_json, old_pregen_clients_json)

# Archive

In [260]:
# Note: Convert old_coll to new stylings and use that as input.
def conversionChecker(new, old):
    if not type(new) == type(old):
        issueRaise(new, old, "Types aren't the same.")
    if isinstance(new, list) and isinstance(old, list):
        if not len(new) == len(old):
            issueRaise(new, old, "Lists aren't the same length.")
        for new_iter, old_iter in zip(new, old):
            conversionChecker(new_iter, old_iter)
    elif isinstance(new, dict) and isinstance(old, dict):
        if not len(new) == len(old):
            issueRaise(new, old, "Dict's aren't the same length.")
        if not new.keys() == old.keys():
            for new_key, old_key in zip(new.keys(), old.keys()):
                if not new_key == old_key:
                    if not len(new_key) == len(old_key):
                        issueRaise(new_key, old_key, "Dict's keys aren't the same.")

        for new_key, old_key in zip(new.keys(), old.keys()):
            if new_key in ['_id', 'exp', 'start_time', 'io', 'runtime', 'cpu', 'FinishedAt', 'StartedAt',
                           'create_time', 'last_update_time', 'message_received_time']:
                pass
            elif new_key in ['actor_id', 'logs', 'id', 'worker_id', 'db_id']:
                if not len(new[new_key]) == len(old[old_key]):
                    issueRaise(new_key, old_key, f"Dict's keys aren't the same. {new[new_key]} | {old[old_key]}")
            else:
                conversionChecker(new[new_key], old[old_key])

    else:
        if not new == old:
            issueRaise(new, old, "Items aren't the same.")